In [1]:
import pandas as pd
import biom
import qiime2
import numpy as np
import os
import skbio

import numpy as np
from scipy.spatial import ConvexHull
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection, Line3DCollection


from qiime2.plugins.emperor.actions import (plot, biplot)
from qiime2.plugins.diversity.actions import (beta_phylogenetic, pcoa, beta)
from qiime2.plugins.feature_table.actions import (rarefy, filter_samples)

from qiime2.plugins.taxa.actions import collapse

import matplotlib.pyplot as plt
from matplotlib.colors import rgb2hex

plt.rcParams['svg.fonttype'] = 'none'

import seaborn as sns

# from skbio.stats.ordination import pcoa
from scipy import stats
%matplotlib inline



### All we will need is a table and associated metadata

In [2]:
qtab = qiime2.Artifact.load('data/table.qza')
meta = pd.read_csv('data/metadata.tsv', sep='\t')
meta.head()

,sample_name,metab-sample-id,age_units,anonymized_name,cage_food_consumption_g,cage_location,cage_notes,cage_number,collection_timestamp,description,...,physical_specimen_location,qiita_study_id,sample_type,scientific_name,sex,taxon_id,title,weight_units,age_bin,age_match
0,11829.A45.179.2,A45_179_2,weeks,A45.179.2,44.2,On shelf,NaN,A45,4/6/18 11:30,Mouse 179 stool collection 2 of 21,...,UCSDMI,11829.0,stool,mouse gut metagenome,male,410661.0,10 week IH and IC,g,12.0,10.5
1,11829.A37.150.21,A37_150_21,weeks,A37.150.21,52.7,In chamber,NaN,A37,6/12/18 10:00,Mouse 150 stool collection 21 of 21,...,UCSDMI,11829.0,stool,mouse gut metagenome,male,410661.0,10 week IH and IC,g,20.0,20.0
2,11829.A35.140.2,A35_140_2,weeks,A35.140.2,37.2,On shelf,Fighting cage,A35,4/6/18 12:00,Mouse 140 stool collection 2 of 21,...,UCSDMI,11829.0,stool,mouse gut metagenome,male,410661.0,10 week IH and IC,g,12.0,10.5
3,11829.A42.168.11,A42_168_11,weeks,A42.168.11,48.2,In chamber,NaN,A42,5/8/18 11:00,Mouse 168 stool collection 11 of 21,...,UCSDMI,11829.0,stool,mouse gut metagenome,male,410661.0,10 week IH and IC,g,16.0,15.0
4,11829.A41.166.2,A41_166_2,weeks,A41.166.2,41.8,In chamber,NaN,A41,4/6/18 11:00,Mouse 166 stool collection 2 of 21,...,UCSDMI,11829.0,stool,mouse gut metagenome,male,410661.0,10 week IH and IC,g,12.0,10.5


### Examine how many observations at each timepoint for each condition we have

We are interested in how IH (intermittent hypoxia) or IC (hypercapnia) alter the microbiome. Our time variable will be given by "age_match" and our group variable will be IH vs. IC vs. Air (control) which in the metadata is called "exposure_type"

In [3]:
pd.DataFrame(meta[['age_match', 'exposure_type']].value_counts()).sort_values(by='age_match')

count
age_match exposure_type       
10.0      IC                16
          Air               15
          IH                15
10.5      IC                16
          IH                16
          Air               15
15.0      IC                16
          IH                16
          Air               15
20.0      IC                16
          IH                16
          Air               13

### Visualize each time point in 3d
Use an appropriate distance metric to generate an ordination. If you are unfamiliar with this step please consult the RPCA tutorial on the gemelli github page, which gives a more detailed explanation of how the RPCA distance metric works and how to use it. Your downstream results will be heavily impacted based on which distance metric you choose to use.

In [4]:
# USE YOUR FAVORITE DISTANCE METRIC TO GENERATE DISTANCE MATRIX & ORDINATION
from qiime2.plugins.gemelli.actions import rpca
result = rpca(qtab)
result.biplot.save(f'data/rpca-biplot.qza')
result.distance_matrix.save(f'data/rpca-dmat.qza')

'data/rpca-dmat.qza'

In [5]:
# QIIME2 WILL AUTOMATICALLY DO THIS WHEN I REPLACE W PLUGIN
tab = qtab.view(biom.Table)
ord = result.biplot.view(skbio.OrdinationResults)